In [1]:
# Import all the necessary libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

#%matplotlib inline

In [5]:
# Read data from the CSV into a dataframe
import os


loc = "/home/angloher/projects/Predicting_Winning_Teams/Datasets" #('Premier_league',5)('Bundesliga',2)('French_league',2)('La_Liga',2)
leagues=[('Scotish_league',(2,4))]
current_season=17

raw_data=[]
for league,divisions in leagues:
    league_data=[]
    for j in range(divisions[0],divisions[1]+1):
        division_data=[]
        DIR = os.path.join(loc,league,str(j))
        for i in range(current_season+1-len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]),current_season+1):
            num1=i
            num2=i+1
            prefix=20
            if num1<0:
                num1=100+i
                num2=num1+1
                prefix=19
                if num1==99:
                    num2=0
            print DIR,'{:02d}-{:02d}.csv'.format(num1,num2),str(len(pd.read_csv(os.path.join(DIR,'{}{:02d}-{:02d}.csv'.format(prefix,num1,num2)))))
            division_data.append(pd.read_csv(os.path.join(DIR,'{}{:02d}-{:02d}.csv'.format(prefix,num1,num2))))
        league_data.append(division_data)
    raw_data.append(league_data)



2
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 94-95.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 95-96.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 96-97.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 97-98.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 98-99.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 99-00.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 00-01.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 01-02.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 02-03.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 03-04.csv 180
/home/angloher/projects/Predicting_Winning_Teams/Datasets/Scotish_league/2 04-05.csv 180
/home/angloher/proj

In [ ]:
# Parse data as time 

def parse_date(date):
    #print date
    if date == '':
        return None
    else:
        try:
            return dt.strptime(str(date), '%d/%m/%y').date()
        except ValueError:
            try: 
                return dt.strptime(str(date), '%d/%m/%Y').date()
            except ValueError:
                try:
                    return dt.strptime(str(date), '%Y-%m-%d').date()
                except ValueError:
                    return None



for league_data in raw_data:
    for division_data in league_data:
        for i in range(len(division_data)):
            division_data[i].Date=division_data[i].Date.apply(parse_date)
    



#Gets all the statistics related to gameplay
                      
all_columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','Div','HS','AS','HST','AST','HC','AC']

playing_stats=[]

for league_data in raw_data:
    playing_data=[]
    for division_data in league_data:
        division_stats=[]
        for i in range(len(division_data)):
            extract_columns=[]
            for column in all_columns:
                if column in division_data[i].columns:
                    extract_columns.append(column)
            division_stats.append(division_data[i][extract_columns])
        playing_data.append(division_stats)
    playing_stats.append(playing_data)


In [ ]:
columns_req = ['B365H','B365D','B365A']

betting_stats=[]
empty_cols=pd.Series([np.nan], index=columns_req)
for league_data in raw_data:
    playing_data=[]
    for division in league_data:
        division_data=[]
        for i in range(len(division)):
            if 'B365H' in division[i].columns:
                division_data.append(division[i][columns_req])
            else:
                division_data.append(pd.DataFrame([[np.nan]*len(columns_req)]*len(division[i]),columns=columns_req,index=np.linspace(0,len(division[i])-1,len(division[i]))))
        playing_data.append(division_data)
    betting_stats.append(playing_data)


In [ ]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            if i>=len(teams[team]):
                teams[team].append(None)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,mw+1):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            if i>=len(teams[team]):
                teams[team].append(None)
            
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    #print teams,mw
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,mw+1):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded,len(teams)/2

def get_gss(playing_stat):
    
    GC,num_games = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% num_games) == 0:
            j = j + 1
        
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


for league_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            print division_stats
            division_stats[i]=get_gss(division_stats[i])


In [ ]:
def get_offensive_stats(playing_stat,home_column,away_column):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i][home_column]
        ATGS = playing_stat.iloc[i][away_column]
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            if i>=len(teams[team]):
                teams[team].append(None)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,mw+1):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def get_devensive_stats(playing_stat,home_column,away_column):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i][home_column]
        HTGC = playing_stat.iloc[i][away_column]
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            if i>=len(teams[team]):
                teams[team].append(None)
            
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    #print teams,mw
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,mw+1):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded,len(teams)/2

def get_additional_stats(playing_stat,home_column,away_column):
    
    GC,num_games = get_devensive_stats(playing_stat,home_column,away_column)
    GS = get_offensive_stats(playing_stat,home_column,away_column)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% num_games) == 0:
            j = j + 1
        
    playing_stat[home_column+'S'] = HTGS
    playing_stat[away_column+'S'] = ATGS
    playing_stat[home_column+'C'] = HTGC
    playing_stat[away_column+'C'] = ATGC
    
    return playing_stat

def set_dummy_stats(stat,col1,col2):
    stat[col1+'S']=np.nan
    stat[col2+'S']=np.nan
    stat[col1+'C']=np.nan
    stat[col2+'C']=np.nan
    return stat

stat_columns=['HS','AS','HST','AST','HC','AC']
for league_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            for j in range(0,len(stat_columns),2):
                if stat_columns[j] in division_stats[i].columns:
                    division_stats[i]=get_additional_stats(division_stats[i],stat_columns[j],stat_columns[j+1])
                    
                else:
                    division_stats[i]=set_dummy_stats(division_stats[i],stat_columns[j],stat_columns[j+1])
            


In [ ]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres,mw,num_teams):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,mw+1):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(num_teams)])
    return matchres_points


def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T,mw,len(teams)

def get_agg_points(playing_stat):
    matchres,mw,num_teams = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres,mw,num_teams)
    HTP = []
    ATP = []
    j = 0
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% (num_teams/2)) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat
    
# Apply to each dataset

for league_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            division_stats[i]=get_agg_points(division_stats[i])


In [ ]:

def get_opponent_points(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        #normalize
        teams[playing_stat.iloc[i].HomeTeam].append(playing_stat.iloc[i].ATP/float(-(-(i+1)/(len(teams)/2))))
        teams[playing_stat.iloc[i].AwayTeam].append(playing_stat.iloc[i].HTP/float(-(-(i+1)/(len(teams)/2))))
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T

def get_opponent_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        #normalize
        teams[playing_stat.iloc[i].HomeTeam].append(playing_stat.iloc[i].ATGS/float(-(-(i+1)/(len(teams)/2))))
        teams[playing_stat.iloc[i].AwayTeam].append(playing_stat.iloc[i].HTGS/float(-(-(i+1)/(len(teams)/2))))
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T

def get_opponent_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        #normalize
        teams[playing_stat.iloc[i].HomeTeam].append(playing_stat.iloc[i].ATGC/float(-(-(i+1)/(len(teams)/2))))
        teams[playing_stat.iloc[i].AwayTeam].append(playing_stat.iloc[i].HTGC/float(-(-(i+1)/(len(teams)/2))))
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T

def get_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        #normalize
        teams[playing_stat.iloc[i].HomeTeam].append(playing_stat.iloc[i].FTHG)
        teams[playing_stat.iloc[i].AwayTeam].append(playing_stat.iloc[i].FTAG)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T

def get_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    mw=-(-len(playing_stat)/(len(teams)/2))
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        #normalize
        teams[playing_stat.iloc[i].HomeTeam].append(playing_stat.iloc[i].FTAG)
        teams[playing_stat.iloc[i].AwayTeam].append(playing_stat.iloc[i].FTHG)
    
    for team in teams:
        for i in range(mw):
            #print i
            #print len(team)
            
            if i>=len(teams[team]):
                teams[team].append(None)
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,mw+1)]).T

def add_form(playing_stat):
    form,mw,num_teams = get_matchres(playing_stat)
    match_res= form.applymap(get_points)
    return match_res,mw,num_teams


def add_form_df(playing_stat,num_last_games):
    
    form,mw,num_teams = add_form(playing_stat)
    opponent_points=get_opponent_points(playing_stat)
    opponent_scored=get_opponent_scored(playing_stat)
    opponent_conceded=get_opponent_conceded(playing_stat)
    scored=get_scored(playing_stat)
    conceded=get_conceded(playing_stat)
    print 'data loaded'
    form_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    form_a_all = [[0] for i in range(num_teams/2)]
    s_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    s_a_all = [[0] for i in range(num_teams/2)]
    c_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    c_a_all = [[0] for i in range(num_teams/2)]
    points_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    points_a_all = [[0] for i in range(num_teams/2)]
    scored_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    scored_a_all = [[0] for i in range(num_teams/2)]
    conceded_h_all = [[0] for i in range(num_teams/2)]  # since form is not available for n MW (n*10)
    conceded_a_all = [[0] for i in range(num_teams/2)]
    j = num_last_games
    
    for i in range((num_teams/2),len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        h_form=[]
        a_form=[]
        h_s=[]
        a_s=[]
        h_c=[]
        a_c=[]
        h_points=[]
        a_points=[]
        h_scored=[]
        a_scored=[]
        h_conceded=[]
        a_conceded=[]
        current_match_week=-(-(i+1)/(num_teams/2))
        for j in range(max(current_match_week-num_last_games,1),current_match_week):
            
            h_form.append(form.loc[ht][j])                    # 0 index is most recent
            a_form.append(form.loc[at][j])                   # 0 index is most recent
            h_s.append(scored.loc[ht][j])
            a_s.append(scored.loc[at][j])
            h_c.append(conceded.loc[ht][j])
            a_c.append(conceded.loc[at][j])
            h_points.append(opponent_points.loc[ht][j])
            a_points.append(opponent_points.loc[at][j])
            h_scored.append(opponent_scored.loc[ht][j])
            a_scored.append(opponent_scored.loc[at][j])
            h_conceded.append(opponent_conceded.loc[ht][j])
            a_conceded.append(opponent_conceded.loc[at][j])   
            
               
        form_h_all.append(h_form)
        form_a_all.append(a_form)
        s_h_all.append(h_s)
        s_a_all.append(a_s)
        c_h_all.append(h_c)
        c_a_all.append(a_c)
        points_h_all.append(h_points)
        points_a_all.append(a_points)
        scored_h_all.append(h_scored)
        scored_a_all.append(a_scored)
        conceded_h_all.append(h_conceded)
        conceded_a_all.append(a_conceded)
    
    #create empty columns
    for i in range(1,num_last_games+1):
        playing_stat['HM' + str(i)+'Points'] = np.nan               
        playing_stat['AM' + str(i)+'Points'] = np.nan
        playing_stat['HM' + str(i)+'Scored'] = np.nan               
        playing_stat['AM' + str(i)+'Scored'] = np.nan
        playing_stat['HM' + str(i)+'Conceded'] = np.nan               
        playing_stat['AM' + str(i)+'Conceded'] = np.nan
        playing_stat['HM' + str(i)+'OppPoints'] = np.nan               
        playing_stat['AM' + str(i)+'OppPoints'] = np.nan
        playing_stat['HM' + str(i)+'OppScored'] = np.nan               
        playing_stat['AM' + str(i)+'OppScored'] = np.nan
        playing_stat['HM' + str(i)+'OppConceded'] = np.nan               
        playing_stat['AM' + str(i)+'OppConceded'] = np.nan
    
    #fill columns
    for i in range(0,len(playing_stat)):
        for game_count in range(1,len(form_h_all[i])+1):
                
                #continue here
            playing_stat['HM' + str(game_count)+'Points'][i] = form_h_all[i][-game_count]                
            playing_stat['AM' + str(game_count)+'Points'][i] = form_a_all[i][-game_count]
            playing_stat['HM' + str(game_count)+'Scored'][i] = s_h_all[i][-game_count]             
            playing_stat['AM' + str(game_count)+'Scored'][i] = s_a_all[i][-game_count]
            playing_stat['HM' + str(game_count)+'Conceded'][i] = c_h_all[i][-game_count]               
            playing_stat['AM' + str(game_count)+'Conceded'][i] = c_a_all[i][-game_count]
            playing_stat['HM' + str(game_count)+'OppPoints'][i] = points_h_all[i][-game_count]               
            playing_stat['AM' + str(game_count)+'OppPoints'][i] = points_a_all[i][-game_count]
            playing_stat['HM' + str(game_count)+'OppScored'][i] = scored_h_all[i][-game_count]               
            playing_stat['AM' + str(game_count)+'OppScored'][i] = scored_a_all[i][-game_count]
            playing_stat['HM' + str(game_count)+'OppConceded'][i] = conceded_h_all[i][-game_count]               
            playing_stat['AM' + str(game_count)+'OppConceded'][i] = conceded_a_all[i][-game_count]
        print i
    
    return playing_stat    
    
# Make changes to df
num_last_games=10
for league_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            print division_stats[i]
            division_stats[i]=add_form_df(division_stats[i],num_last_games)
            


In [ ]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP',
        'Div' ]

playing_stat=playing_stats
for i in range(1,num_last_games+1):
    cols.append('HM' + str(i)+'Points')
    cols.append('AM' + str(i)+'Points')
    cols.append('HM' + str(i)+'Scored')               
    cols.append('AM' + str(i)+'Scored')
    cols.append('HM' + str(i)+'Conceded')               
    cols.append('AM' + str(i)+'Conceded')
    cols.append('HM' + str(i)+'OppPoints')               
    cols.append('AM' + str(i)+'OppPoints')
    cols.append('HM' + str(i)+'OppScored')               
    cols.append('AM' + str(i)+'OppScored')
    cols.append('HM' + str(i)+'OppConceded')               
    cols.append('AM' + str(i)+'OppConceded')
    

stat_columns=['HSS', 'ASS', 
              'HSC', 'ASC', 'HSTS', 'ASTS', 'HSTC','ASTC', 
              'HCS', 'ACS', 'HCC', 'ACC']
for league_stats in playing_stat:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            extract_cols=list(cols)
            for j in range(0,len(stat_columns),2):
                if stat_columns[j] in division_stats[i].columns:
                    extract_cols.append(stat_columns[j])
                    extract_cols.append(stat_columns[j+1])
            division_stats[i]=division_stats[i][extract_cols]


In [ ]:
#create standings file
import collections




league_count=0
for league_stats in playing_stats:
    division_count=1
    for division_stats in league_stats:
        teams={}
        for i in range(len(division_stats)):
            for j in division_stats[i].groupby('HomeTeam').mean().T.columns:
                teams[j] = [0]
        sorted_teams=collections.OrderedDict(sorted(teams.items()))
        for i in range(len(division_stats)):
            for team in sorted_teams:
                HT=division_stats[i].loc[division_stats[i]['HomeTeam'] == team].index
                AT=division_stats[i].loc[division_stats[i]['AwayTeam'] == team].index
                if len(HT)==0 and len(AT)==0:
                    sorted_teams[team].append(0)
                elif max(HT)>max(AT):
                    points=division_stats[i].iloc[max(HT)]['HTP']
                    result=division_stats[i].iloc[max(HT)]['FTR']
                    if result=='H':
                        points+=3
                    elif result=='D':
                        points+=1
                    sorted_teams[team].append(points)
                elif max(AT)>max(HT):
                    points=division_stats[i].iloc[max(AT)]['ATP']
                    result=division_stats[i].iloc[max(AT)]['FTR']
                    if result=='A':
                        points+=3
                    elif result=='D':
                        points+=1
                    sorted_teams[team].append(points)
        keys=[]
        vals=[]
        for team in sorted_teams:
            keys.append(team)
            vals.append(sorted_teams[team])
        csv_standings=pd.DataFrame(vals,index=keys,columns=np.linspace(2000+current_season-len(division_stats),2000+current_season,len(division_stats)+1))
        csv_standings.index.name='Team'
        csv_standings.to_csv(os.path.join(loc,leagues[league_count][0],str(division_count),'Results','Standings.csv'))
        division_count+=1
    league_count+=1

In [ ]:


def get_last(playing_stat, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    return playing_stat

league_count=0
for league_stats in playing_stats:
    division_count=1
    for division_stats in league_stats:
        Standings = pd.read_csv(os.path.join(loc,leagues[league_count][0],str(division_count),'Results','Standings.csv'))
        division_count+=1
        Standings.set_index(['Team'], inplace=True)
        division_stats[0]['HomeTeamLP']=[np.nan]*len(division_stats[0])
        division_stats[0]['AwayTeamLP']=[np.nan]*len(division_stats[0])
        for i in range(0,len(division_stats)):
            division_stats[i]=get_last(division_stats[i],Standings,i)
    league_count+=1


In [ ]:
def get_mw(playing_stat):
    
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    j = 1
    MatchWeek = []
    for i in range(len(playing_stat)):
        MatchWeek.append(j)
        if ((i + 1)% (len(teams)/2)) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat


for league_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            division_stats[i]=get_mw(division_stats[i])


In [ ]:

    
cols = ['HTGS','HTGC','ATGS','ATGC','HTP','ATP','HSS', 'ASS', 'HSC', 'ASC', 'HSTS', 'ASTS', 'HSTC', 'ASTC', 'HCS', 'ACS', 'HCC', 'ACC']
for division_stats in playing_stats:
    for division_stats in league_stats:
        for i in range(len(division_stats)):
            divis=division_stats[i].MW-1
            divis=divis.replace(0,1)
            for col in cols:
                division_stats[i][col] = division_stats[i][col] / divis
                

In [ ]:
league_count=0

for league_stats in betting_stats:
    division_count=leagues[league_count][1][0]
    for division_stats in league_stats:
        playing_stat = pd.concat(division_stats, ignore_index=True)
        playing_stat.to_csv(os.path.join(loc,leagues[league_count][0],str(division_count),'Results','betting_odds.csv'))
        division_count+=1
    league_count+=1


In [ ]:
league_count=0


for league_stats in playing_stats:
    division_count=leagues[league_count][1][0]
    for division_stats in league_stats:
        
        playing_stat = pd.concat(division_stats, ignore_index=True)
        playing_stat.to_csv(os.path.join(loc,leagues[league_count][0],str(division_count),'Results','final_dataset.csv'))
        division_count+=1
    league_count+=1
